<a href="https://colab.research.google.com/github/vanessajen/Sam-tracker-v4/blob/main/Tiktok_Reeditor_V12_Stable_Pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TikTok Reeditor V12 Stable Pro (Updated)
# By Sam for Jenny ✭ (Async + Scrape US/UK TikTok Shop)

# === INSTALL LIBRARIES ===
!pip install yt-dlp playwright nest_asyncio moviepy
!playwright install chromium

# === CONNECT GOOGLE DRIVE ===
from google.colab import drive
import shutil

drive.mount('/content/drive')

# === IMPORT LIBRARIES ===
import requests
import pandas as pd
from datetime import datetime
import os
import json
import subprocess
import random
import time
import re
import asyncio
from typing import List, Dict, Any
from playwright.async_api import async_playwright, Page
from moviepy.editor import VideoFileClip, concatenate_videoclips, AudioFileClip, TextClip, CompositeVideoClip
import nest_asyncio
nest_asyncio.apply()

# === CONFIG ===
HISTORY_FOLDER = "sales_history"
EXPORT_CSV_PATH = "tshirt_growth_report.csv"
VIDEO_DOWNLOAD_FOLDER = "downloaded_videos"
FINAL_VIDEO_OUTPUT = "reup_video_today_trendy_cut_scene.mp4"
FINAL_VIDEO_WITH_MUSIC = "reup_video_final_with_music.mp4"
THUMBNAIL_IMAGE = "thumbnail.jpg"
TRENDY_MUSIC_FOLDER = "/content/drive/MyDrive/music"
REUP_DRIVE_FOLDER = "/content/drive/MyDrive/Reup_Videos"
MAX_PRODUCTS = 60

# === TELEGRAM CONFIG ===
TELEGRAM_BOT_TOKEN = "8118142644:AAEVkSRX8ACgeCsCS9h3IjC8Kraf_v_poPY"
TELEGRAM_CHAT_ID = "1157657879"

# === CREATE FOLDER ===
os.makedirs(HISTORY_FOLDER, exist_ok=True)
os.makedirs(VIDEO_DOWNLOAD_FOLDER, exist_ok=True)
os.makedirs(TRENDY_MUSIC_FOLDER, exist_ok=True)
os.makedirs(REUP_DRIVE_FOLDER, exist_ok=True)

# === SMART ENGLISH HOOKS ===
HOOKS = [
    "You won't believe this T-shirt!",
    "This design is taking over the US!",
    "Top trending tee today ✨",
    "Everyone is obsessed with this shirt!",
    "The viral T-shirt you must have!",
    "Don't miss this amazing deal!",
    "The hardest shirt doesn't exist... or does it?",
    "Sorry for those who bought it yesterday – today it's on SALE!"
]

# === FUNCTION: Send telegram ===
def send_to_telegram(message):
    url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
    payload = {"chat_id": TELEGRAM_CHAT_ID, "text": message, "parse_mode": "HTML"}
    requests.post(url, data=payload)

# === (Scraper class giữ nguyên, như cũ) ===

# === FUNCTION: Video Editing ===
def edit_video():
    try:
        clips = []
        for filename in os.listdir(VIDEO_DOWNLOAD_FOLDER):
            if filename.endswith(".mp4"):
                clip = VideoFileClip(os.path.join(VIDEO_DOWNLOAD_FOLDER, filename)).subclip(0, 7)
                clips.append(clip)
        if not clips:
            print("No clips found to edit.")
            return

        final_clip = concatenate_videoclips(clips)

        # Add trendy music
        music_files = [os.path.join(TRENDY_MUSIC_FOLDER, f) for f in os.listdir(TRENDY_MUSIC_FOLDER) if f.endswith(".mp3")]
        if music_files:
            music_clip = AudioFileClip(random.choice(music_files)).subclip(0, final_clip.duration)
            final_clip = final_clip.set_audio(music_clip)

        # Add hook text overlays
        hook_text = random.choice(HOOKS)
        txt_clip = TextClip(hook_text, fontsize=48, color='white', font='Arial-Bold')\
                    .set_position('center').set_duration(final_clip.duration)
        final_with_text = CompositeVideoClip([final_clip, txt_clip])

        # Export final video
        final_with_text.write_videofile(FINAL_VIDEO_WITH_MUSIC, codec='libx264', audio_codec='aac')
        print("Final re-up video created!")
    except Exception as e:
        print(f"Error editing video: {e}")

# === MAIN EXECUTION ===

async def main():
    today_str = datetime.today().strftime('%Y-%m-%d')
    today_path = f"{HISTORY_FOLDER}/{today_str}.json"

    if os.path.exists(today_path):
        with open(today_path) as f:
            today_data = json.load(f)
        send_to_telegram("✅ Found today's sales data. Proceeding...")
    else:
        send_to_telegram("🌍 No today's data found. Scraping TikTok Shop products!")
        scraper = TikTokShopScraper()
        today_data = await scraper.scrape()
        with open(today_path, 'w') as f:
            json.dump(today_data, f, ensure_ascii=False, indent=2)
        send_to_telegram(f"✅ Scraped {len(today_data)} products successfully!")

    send_to_telegram("🎥 Start downloading and filtering videos!")

    # (Placeholder for download code)

    send_to_telegram("✅ Downloaded and filtered videos!")

    # Editing re-up video
    send_to_telegram("🎬 Editing video...")
    edit_video()

    if os.path.exists(FINAL_VIDEO_WITH_MUSIC):
        shutil.copy(FINAL_VIDEO_WITH_MUSIC, f"{REUP_DRIVE_FOLDER}/{FINAL_VIDEO_WITH_MUSIC}")
        send_to_telegram("📂 Final re-up video uploaded to Drive!")
    else:
        send_to_telegram("⚠️ Final video not found. Cannot upload.")

await main()
